# Meeting Transcript Summarization

In [9]:
!pip install simplet5

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torchsummary import summary

In [12]:
df = pd.read_json('../input/meeting-transcript-summarization-dataset/train.json')
df.head()

,id,summary,dialogue
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella...."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...


In [13]:
df.shape

(14732, 3)

In [14]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"summary": "target_text", "dialogue": "source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: Amanda: I baked cookies. Do you wa...,Amanda baked cookies and will bring Jerry some...
1,summarize: Olivia: Who are you voting for in t...,Olivia and Olivier are voting for liberals in ...
2,"summarize: Tim: Hi, what's up?\r\nKim: Bad moo...",Kim may try the pomodoro technique recommended...
3,"summarize: Edward: Rachel, I think I'm in ove ...",Edward thinks he is in love with Bella. Rachel...
4,summarize: Sam: hey overheard rick say someth...,"Sam is confused, because he overheard Rick com..."
...,...,...
14727,summarize: Romeo: You are on my ‘People you ma...,Romeo is trying to get Greta to add him to her...
14728,summarize: Theresa: <file_photo>\r\nTheresa: <...,Theresa is at work. She gets free food and fre...
14729,summarize: John: Every day some bad news. Japa...,Japan is going to hunt whales again. Island an...
14730,summarize: Jennifer: Dear Celia! How are you d...,Celia couldn't make it to the afternoon with t...


In [15]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((11785, 2), (2947, 2))

In [16]:
from simplet5 import SimpleT5
import gc
gc.collect()

92

In [17]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=400, 
            target_max_token_len=200, 
            batch_size=8, max_epochs=10, use_gpu=True)